In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, sequence, lit, year, month, dayofmonth, quarter, dayofweek, when
from pyspark.sql.types import BooleanType, DateType
from pyspark.sql.functions import *
 
 
spark = SparkSession.builder.getOrCreate()
 

spark.conf.set(
    "",
    ""
)

 
bronze_path = "abfss://traffic@hellooodb.dfs.core.windows.net/raw_data"
 
schema = """
    vehicle_id STRING,
    vehicle_count INT,
    speed INT,
    fuel_level DOUBLE,
    engine_temp DOUBLE,
    status STRING,
    road_type STRING,
    direction STRING,
    timestamp LONG,
    location STRING,
    city STRING,
    alert STRING
"""
 
df = spark.read.format("delta").load(bronze_path)
 
df_bronze = (
    df
    .select(F.from_json(F.col("json_value").cast("string"), schema).alias("data"))
    .select("data.*")
    .filter(col("status") != "stopped")
    .withColumn("location", F.regexp_replace(col("location"), r"[\[\]()]", ""))
    .withColumn("speed", when(col("speed") > 200, 120).otherwise(col("speed")))
    .withColumn("fuel_level", when(col("fuel_level") < 0, 0.0).otherwise(col("fuel_level")))
    .withColumn("engine_temp", when(col("engine_temp") > 150, 120.0).otherwise(col("engine_temp")))
    .withColumn("event_ingestion_time", F.to_date(current_timestamp()))
    .drop("timestamp")
)
 
df_bronze.write.mode("overwrite").saveAsTable("traffic.traffic_insights.Analyze_vehicle_silver")

In [0]:
display(df_bronze)